In [1]:
import torch
import os
import evaluation

from importlib import reload
reload(evaluation)

<module 'evaluation' from '/home/erik/Desktop/WiSe2023_24/xAI-Proj-M/xAI-Proj-M/lowdim/evaluation.py'>

# Load and Check Training results
In this directory the model trainer class stores all embeddings and predictions that were prodcued per epoch.
That way we can monitor how the (2d) embeddings and the class centroids change per epoch.
In this notebook those artifacts get loaded for a sample training run and check for plausibility/correctness.

In [2]:
save_model_subfolder_path = os.getcwd() + "/savedlowdimmodels/Skipped18LayerForLowDimEmbbeding_20240125_1815"

In [3]:
epoch_0_results_path = save_model_subfolder_path + "/epoch_0"

In [4]:
train_embeddings = torch.load(epoch_0_results_path + "/train_embeddings.pt")
print("Nr of training embeddings produced in epoch 1 while training (excluded val samples): " + str(len(train_embeddings)) )

Nr of training embeddings produced in epoch 1 while training (excluded val samples): 45000


### Describe data stored for epoch 0

In [5]:
print(evaluation.describe_epoch_result(epoch_0_results_path))

Describe contents in /home/erik/Desktop/WiSe2023_24/xAI-Proj-M/xAI-Proj-M/lowdim/savedlowdimmodels/Skipped18LayerForLowDimEmbbeding_20240125_1815/epoch_0
------train_embeddings-------
path: /home/erik/Desktop/WiSe2023_24/xAI-Proj-M/xAI-Proj-M/lowdim/savedlowdimmodels/Skipped18LayerForLowDimEmbbeding_20240125_1815/epoch_0/train_embeddings.pt
type of data: <class 'list'>
length of data: 45000
type of elements: <class 'list'>
first element: [0.6281577348709106, 0.3842242658138275]
length of single sample: 2
------train_labels-------
path: /home/erik/Desktop/WiSe2023_24/xAI-Proj-M/xAI-Proj-M/lowdim/savedlowdimmodels/Skipped18LayerForLowDimEmbbeding_20240125_1815/epoch_0/train_labels.pt
type of data: <class 'list'>
length of data: 45000
type of elements: <class 'torch.Tensor'>
first element: 7
shape of single sample: torch.Size([])
------train_predictions-------
path: /home/erik/Desktop/WiSe2023_24/xAI-Proj-M/xAI-Proj-M/lowdim/savedlowdimmodels/Skipped18LayerForLowDimEmbbeding_20240125_1815